In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
df = pd.read_parquet("/beegfs/ws/0/s4610340-energy_behavior/yahor/kaggle-predict_energy_behavior_of_prosumers/data/processed/train/make_features/df_features.parquet", engine="fastparquet")
df = df.loc[df["is_consumption"]==0]
df["target_per_capacity"] = df["target"] / df["installed_capacity"]
df

,county,is_business,product_type,is_consumption,datetime,date,dayofyear,hour,day,weekday,...,diff_168h_windspeed_10m_forecast,diff_168h_10_metre_u_wind_component_historical,diff_168h_10_metre_u_wind_component_forecast,diff_168h_10_metre_v_wind_component_historical,diff_168h_10_metre_v_wind_component_forecast,diff_168h_humidity_historical,diff_168h_humidity_forecast,diff_168h_surface_solar_radiation_downwards_forecast,target,target_per_capacity
row_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,0,2021-09-01 00:00:00,2021-09-01,244,0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.713,inf
122,0,0,1,0,2021-09-01 01:00:00,2021-09-01,244,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.132,inf
244,0,0,1,0,2021-09-01 02:00:00,2021-09-01,244,2,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.490,inf
366,0,0,1,0,2021-09-01 03:00:00,2021-09-01,244,3,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.496,inf
488,0,0,1,0,2021-09-01 04:00:00,2021-09-01,244,4,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.149,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017818,14,1,2,0,2023-05-31 19:00:00,2023-05-31,151,19,31,3,...,3.820864,-0.298044,4.869689,0.113850,-1.154504,-2.619718,4.432365,11.855307,35.026,0.142382
2017948,14,1,2,0,2023-05-31 20:00:00,2023-05-31,151,20,31,3,...,2.733154,-0.665162,4.048730,0.498659,-1.076261,-3.987781,2.489141,16.528545,14.867,0.060435
2018078,14,1,2,0,2023-05-31 21:00:00,2023-05-31,151,21,31,3,...,1.804359,0.315522,3.727382,0.606830,-0.177199,-4.616146,3.756591,10.491406,1.865,0.007581


In [5]:
def explore_feature_vs_target(x: str, win=10, step=10, val_thr=0.0):
    series = df.set_index(df[x] / df[x].max()).sort_index()["target_per_capacity"]
    series = series.loc[series.index>val_thr]
    series_rolling_median = series.rolling(win).median()[::step]
    series_rolling_q75 = series.rolling(win).quantile(0.75)[::step]
    series_rolling_q25 = series.rolling(win).quantile(0.25)[::step]

    fig = go.Figure(
        [
            go.Scattergl(
                x=series_rolling_median.index, 
                y=series_rolling_median, 
                mode="markers", 
                hoverinfo="skip",
            ),
            go.Scatter(
                x=series_rolling_q75.index,
                y=series_rolling_q75,
                mode='lines',
                marker=dict(color="#444"),
                line=dict(width=0),
                showlegend=False
            ),
            go.Scatter(
                x=series_rolling_q25.index,
                y=series_rolling_q25,
                marker=dict(color="#444"),
                line=dict(width=0),
                mode='lines',
                fillcolor='rgba(68, 68, 68, 0.3)',
                fill='tonexty',
                showlegend=False
            )
        ]
    )

    return fig


explore_feature_vs_target("surface_solar_radiation_downwards_forecast", 150, 100)

In [20]:
explore_feature_vs_target("snowfall_historical", 100, 50, 0.0)

In [9]:
df["rain_forecast"].clip(0).describe()

count    1.007457e+06
mean     4.465757e-05
std      1.662766e-04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      9.827024e-06
max      3.942772e-03
Name: rain_forecast, dtype: float64

In [23]:
(df["total_precipitation_forecast"] - df["snowfall_forecast"]*334/(100*1000)).describe()

count    1.007457e+06
mean     8.307369e-05
std      2.075521e-04
min     -5.795658e-06
25%      0.000000e+00
50%      7.737336e-06
75%      6.017493e-05
max      3.942772e-03
dtype: float64